In [1]:
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import abs as psabs
from pyspark.sql.functions import from_utc_timestamp

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1588557304199_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
df = sqlContext.read.csv('s3a://sparkdemonstration/nyc-train.csv', header=True, inferSchema=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Using a smaller dataset of 10M rows
df = df.limit(10000000)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- key: timestamp (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- passenger_count: integer (nullable = true)

In [5]:
# Keep only relevant columns
usecols = [c for c in df.columns if c != 'key']
df = df.select(*usecols)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Removing outliers - filter fares that are negative or too large.
df = df.filter(df['fare_amount'] > 0)
df = df.filter(df['fare_amount'] <= 250)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# Removing outliers - filter passenger counts that are negative or too large.
df = df.filter(df['passenger_count'] > 0)
df = df.filter(df['passenger_count'] <= 10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# Cleaning data - NYC lies between 73 and 75 degrees West, and 40 and 42 degrees north.
# Remove trips that have pickup or dropoff coordinates outside NYC.

LEFT, RIGHT, BOTTOM, TOP =  -75, -73, 40, 42
df = df.filter(df['pickup_longitude'] >= LEFT)
df = df.filter(df['pickup_longitude'] <= RIGHT)
df = df.filter(df['pickup_latitude'] <= TOP)
df = df.filter(df['pickup_latitude'] >= BOTTOM)
df = df.filter(df['dropoff_latitude'] >= BOTTOM)
df = df.filter(df['dropoff_latitude'] <= TOP)
df = df.filter(df['dropoff_longitude'] <= RIGHT)
df = df.filter(df['dropoff_longitude'] >= LEFT)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# Approximate the physical distance between pickup and dropoffs using the L1 norm.
# (Remember that Manhattan is in NYC!)

x1 = df['pickup_longitude']
y1 = df['pickup_latitude']
x2 = df['dropoff_longitude']
y2 = df['dropoff_latitude']

df = df.withColumn('l1', psabs(x1 - x2) + psabs(y1 - y2))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# Prices may depend on time of day! (Note that original timestamps are in UTC)
df = df.withColumn('datetime', df['pickup_datetime'].substr(0, 19))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# Convert the timestamps to New York time.
df = df.withColumn('NYTime', from_utc_timestamp('datetime', 'EST'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# Extract features from the timestamps.

from pyspark.sql.functions import year, month, dayofweek, hour
df = df.withColumn('year', year(df['NYTime']))
df = df.withColumn('month', month(df['NYTime']))
df = df.withColumn('day', dayofweek(df['NYTime']))
df = df.withColumn('hour', hour(df['NYTime']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
df.select('year', 'month', 'day', 'hour', 'NYTime').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+---+----+-------------------+
|year|month|day|hour|             NYTime|
+----+-----+---+----+-------------------+
|2010|    2|  2|  17|2010-02-08 17:14:00|
|2010|    6|  6|  10|2010-06-18 10:26:00|
|2014|   11|  3|  15|2014-11-18 15:33:40|
|2011|    6|  3|  15|2011-06-14 15:38:00|
|2011|    6|  3|   2|2011-06-21 02:08:16|
|2009|    2|  3|  14|2009-02-10 14:17:09|
|2009|    8|  6|  17|2009-08-07 17:03:00|
|2009|   10|  3|  21|2009-10-13 21:38:00|
|2012|    7|  6|  10|2012-07-13 10:01:23|
|2013|   11|  5|  19|2013-11-21 19:09:00|
|2014|   10|  5|   6|2014-10-30 06:08:00|
|2012|    2|  4|  23|2012-02-15 23:11:14|
|2012|   12|  3|  10|2012-12-04 10:46:10|
|2013|    5|  7|  18|2013-05-11 18:57:00|
|2010|   12|  4|   1|2010-12-08 01:27:46|
|2011|    3|  2|  21|2011-03-28 21:46:00|
|2014|    1|  2|  17|2014-01-13 17:43:00|
|2014|    4|  4|  13|2014-04-30 13:09:00|
|2015|    3|  3|  13|2015-03-10 13:33:13|
|2012|    1|  3|  10|2012-01-10 10:53:00|
+----+-----+---+----+-------------

In [14]:
# Drop rows containing null values
df = df.dropna()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
featureCols = [c for c in df.columns if c not in ['datetime', 'NYTime', 'pickup_datetime']]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=featureCols, outputCol='features')
df = assembler.transform(df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
train, test = df.randomSplit([0.66, 0.33])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
from pyspark.ml.regression import LinearRegression

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
lr = LinearRegression(featuresCol='features', labelCol='fare_amount')
model = lr.fit(train)
summary = model.evaluate(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
summary.rootMeanSquaredError

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8.04980932763291e-12

In [21]:
summary.r2

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1.0